<a href="https://colab.research.google.com/github/soumo99/AI_ML_Projects/blob/main/stock_trend_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data

In [ ]:
start = '2010-01-01'
end = '2019-12-31'

#Scraping the data from yahoo finance website
df = data.DataReader('AAPL','yahoo', start, end)

df.head()

The project is mainly based on the closing price for the particular date 

In [ ]:
pip install pandas-datareader

In [ ]:
pip install --upgrade pandas

In [ ]:
pip install --upgrade pandas-datareader

In [ ]:
df.tail()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Date','Adj Close'],axis=1)
df.head()

In [ ]:
plt.plot(df.Close)

In [ ]:
df

In [ ]:
#Calculating the 100 days moving average(the averages of a time frame) so the firsr 100 days(so it will take the 100 days closing price and will find the average) it will be null 
ma_100 = df.Close.rolling(100).mean()
ma_100

In [ ]:
#Plotting the graph for 100 days moving average 
plt.figure(figsize = (12,6))
plt.plot(df.Close)
plt.plot(ma_100,'r')

In [ ]:
#Calculating the 200 days moving average so the firsr 100 days it will be null 
ma_200 = df.Close.rolling(200).mean()
ma_200

In [ ]:
#Plotting the graph for 200 days moving average 
plt.figure(figsize = (12,6))
plt.plot(df.Close)
plt.plot(ma_200,'r')

In [ ]:
#Plotting the combined graph of 100 and 200 days moving average 
plt.figure(figsize = (12,6))
plt.plot(df.Close)
plt.plot(ma_100,'g')
plt.plot(ma_200,'r')


In [ ]:
df.shape

The main data will be the closing column 

In [ ]:
#Splitting the data into 70% training data and 30% testing data
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])
print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_training.head()

In [ ]:
data_testing.head()

In [ ]:
#for stack LSTM model we have scale down the data 
#we will scale the data between 0 and 1

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

In [ ]:
#Fitting the datas 
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [ ]:
#Divide the data into x train and y train 
#the next days price will be totally dependent on the previous days data and when again the next days data will be calculated the first days data will be deducted so like this the whole process will carry on  
x_train = [] #previous days data (featured class)
y_train = [] #next days data (predicted class)

#here the steps are starting from 100 

for i in range(100,data_training.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i,0])
    
    
x_train, y_train = np.array(x_train),np.array(y_train)

Building the Machine Learning Model 

In [ ]:
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

The 4 layers of the LSTM Model

In [ ]:
#FIRST LAYER
model = Sequential()
model.add(LSTM(units=50,activation='relu',return_sequences=True,
               input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

#SECOND LAYER
model = Sequential()
model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

#THIRD LAYER
model = Sequential()
model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.4))

#LAST LAYER
model = Sequential()
model.add(LSTM(units=120,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
#For loss mean squared error is being used because we are doing time series analysis(for predicting the next values we need to calculate the previous values) and for that mean squared error is being used 
model.compile(optimizer='adam',loss = 'mean_squared_error')
model.fit(x_train,y_train,epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_training.tail(100)

In [ ]:
past_100_days = data_training.tail(100)

In [ ]:
final_df = past_100_days.append(data_testing, ignore_index=True)

In [ ]:
final_df.head()

Scaling the datas by using the Min Max Scaler because the above table datas are not scaled down 

In [ ]:
#So all the rows are being scaled down between 0 and 1
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100,input_data.shape[0]):
  x_test.append(input_data[i-100:i])
  y_test.append(input_data[i,0])


In [ ]:
x_test, y_test = np.array(x_test),np.array(y_test)
print(x_test.shape)
print(y_test.shape)

Making Predictions 

In [ ]:
y_predicted = model.predict(x_test)


In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
#Calculating the factor by which all the valuea are scaled down 
scaler.scale_

In [ ]:
scale_factor = 1/0.02099517
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test,'b',label = 'Original Price')
plt.plot(y_predicted,'r',label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()